
<h1 style="color: #4d2c6d;">PROBLEM DEFINITON AND SOLUTION APROACH</h1>

In first part, there is Personality Classification Dataset which contains 10000 samples with 60 attributes and 1 class label. We aim to classify different personality types of people using normal and weighted nearest neighbor algorithm with original and normalized dataset. In addition, we analyse our predictions using accuracy, precision and recall metrics.
<br>
In second part,there is Energy Efficiency Estimation Dataset which contains less than 1000 samples with 8 attributes and 2 class labels named Heating Load and Cooling Load. For this part, we again use normal and weighted nearest neighbor algorithm with original and normalized dataset. To analyse our predictions, we use Mean Absolute Error which is regression performance metric.
<br><br>
For both parts , since the Personality Classification Dataset contains too much samples and we calculate for 5 folds with 5 k-values for Normal and Weighted KNN for Non-Normalized and Normalized Data, it simply takes a long time for running. Since KNN algrithm uses distance arrays to predict the test samples , before running KNN algortihm, we calculated the distance arrays of all test samples for all 5-folds so that our running time 10 times faster. The algorithm is , for each test sample in test data, we calculate the euclidean distance between every training sample and test sample. While processing normal and weighted KNN , we just uses distance arrays to predict the test samples for 5 k-values in a very short time.


In [1]:
#BBM409 Assignment 1 
#b21946242 Numan Kafadar
#b21946198 Umut Güngör

#importing necessary libraries
import numpy as np
import pandas as pd
import math
import time
from random import randint

In [57]:
#fischer-yates algorithm to shuffle the dataset
def shuffle(arr):
    n=len(arr)
    arr=arr.tolist()
    for i in range(n-1,0,-1):
        j = randint(0,i+1)
        arr[i],arr[j] = arr[j],arr[i]
    return np.asarray(arr)


#normalization method for training dataset
def normalization(data_set):
    #geting min and max values of each columns
    min_set = data_set.min(axis=0)
    max_set = data_set.max(axis=0)
    num_rows, num_cols = data_set.shape
    
    new_data_set = np.zeros((num_rows, num_cols)).astype(dtype="float64")
    
    #min-max normalization
    for i in range(num_rows):
        for j in range(num_cols):
            a1 = data_set[i, j] - min_set[j]
            a2 = max_set[j] - min_set[j]
            new_data_set[i, j] = a1 / a2
    
    return new_data_set

#splitting data into 5 parts using kFold
def kFold(data,labels,k=5):
    size=len(data)
    n=int(size/k)  #length of test samples
    train_test_data=[]
    
    for i in range(k):
        x_train=np.concatenate([data[:(i*n)],data[((i+1)*n):]])
        x_test=np.concatenate([labels[:(i*n)],labels[((i+1)*n):]])
        y_train=data[(i*n):((i+1)*n)]
        y_test=labels[(i*n):((i+1)*n)]
        train_test_data.append([x_train,x_test,y_train,y_test])
        
    return train_test_data

#euclidean distance algorithm using numpy functions
def fast_euclidean_distance(x,y):
    return np.power(np.sum(np.power((x-y),2)),0.5,dtype="float64")

#calculating distance arrays for eacht test sample before KNN prediction
def calculate_distances(train_data,test_data, labels):
    result = []
    for test_vector in test_data:
        nearest_neighbors = []
        idx = 0
        for row in train_data:
            dist = fast_euclidean_distance(test_vector, row)
            nearest_neighbors.append((dist, labels[idx]))
            idx += 1
            
        nearest_neighbors.sort(key=lambda t: t[0]) #sort the distances
        result.append(nearest_neighbors)
        
    return result


#KNN algorithm for prediction
def kNN(distances,k,kNNType="Normal"):
    predictions=[]
    #weighted KNN
    if kNNType=="Weighted":
        for d in distances:
            #get first k elements of the distance array
            results = [row[1] for row in d[:k]] 
            dists=[row[0] for row in d[:k]] 
            neigs={}
            
            for i in range(len(results)):
                #for each distance value, get its weight as 1/d
                weight=1.0/dists[i]
                if results[i] in neigs:
                    neigs[results[i]]+=weight
                else:
                    neigs[results[i]]=weight
            predictions.append(max(neigs, key=neigs.get))
           
    #normal KNN
    else:
        for d in distances:
            #get first k elements of the distance array
            results = [row[1] for row in d[:k]]
            pred = max(set(results), key=results.count)
            predictions.append(pred)

    return predictions

#calculating accuracy, precision and recall metrics using confusion matrix
def get_statics(actualVals, predVals ,classes): 
    #confusion matrix
    conf_matrix = pd.DataFrame(
    np.zeros((len(classes), len(classes)),dtype=int),
                    index=classes, columns=classes)
    for i, j in zip(actualVals ,predVals):
        conf_matrix.loc[i, j] += 1
    
    #getting precision and recall using confusion matrix
    precision,recall=calculate_metrics(conf_matrix.to_numpy())

    #calculating accuracy
    correct = 0
    for i in range(len(actualVals)):
        if actualVals[i] == predVals[i]:
            correct += 1
            
    accuracy=correct / float(len(actualVals)) 

    return accuracy*100, precision*100, recall*100

# a method for getting precision and recall metrics using confusion matrix
def calculate_metrics(metrics_matrix):
    precision = 0.0
    recall = 0.0
    for i in range(16):
        precision += metrics_matrix[i, i] / np.sum(metrics_matrix[i])
        recall += metrics_matrix[i, i] / np.sum(metrics_matrix[:, i])
        
    precision /= 16
    recall /= 16
    
    return  precision , recall


#Mean Absolute Error calculation method
def mae(actuals ,preds):
    dif=0.0
    for i in range(len(actuals)):
        dif+=abs(actuals[i]-preds[i])
    return dif/len(actuals)


In [ ]:
# a function to get misclassified items to comment why they are misclassified 
"""
def get_misclassified_items(test_data,actuals,preds):
    res=[]
    for i in range(len(actuals)):
        if actuals[i] != preds[i]:
            res.append([test_data[i],actuals[i],preds[i]])
    return res
"""

In [14]:
#data exploring
df = pd.read_csv("subset_16P.csv", encoding="cp1252")
df.head()


,Response Id,You regularly make new friends.,You spend a lot of your free time exploring various random topics that pique your interest,Seeing other people cry can easily make you feel like you want to cry too,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",You prefer to completely finish one project before starting another.,You are very sentimental.,You like to use organizing tools like schedules and lists.,...,You believe that pondering abstract philosophical questions is a waste of time.,"You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.",You know at first glance how someone is feeling.,You often feel overwhelmed.,You complete things methodically without skipping over any steps.,You are very intrigued by things labeled as controversial.,You would pass along a good opportunity if you thought someone else needed it more.,You struggle with deadlines.,You feel confident that things will work out for you.,Personality
0,35874,-1,0,-1,1,-1,-2,-2,0,-1,...,0,3,0,0,0,0,1,-1,0,ENTP
1,42624,0,0,1,0,0,0,-1,0,0,...,0,2,0,0,0,0,-1,-3,2,INTP
2,55199,0,0,-2,-1,2,-2,0,0,-1,...,0,0,0,1,0,0,3,0,0,ESTP
3,52983,0,0,0,1,-2,-1,0,0,1,...,1,1,0,-1,0,-1,2,-2,0,ENTP
4,22864,0,0,2,1,0,-2,-1,0,1,...,1,-2,0,1,0,0,0,-2,2,ENFJ


In [16]:
arr = np.array(df)

#shuffle the numpy array
arr=shuffle(arr)

#splitting labels and features
x = arr[:, 1:-1].astype(dtype="int32")
y = arr[:, -1:]
y = y.reshape(len(y),)


In [18]:
#split the data into 5 fold
folds=kFold(x,y)

#First , getting all the distance arrays to process KNN
foldsResult=[]
fnum=1
print("Non-Normalized Data ,Getting Distance arrays...")
for fold in folds:
    st=time.time()
    dist=calculate_distances(fold[0],fold[2],fold[1])
    et=time.time()
    passed=et-st
    print("For fold {} , prediction time = {} ".format(fnum,passed))
    foldsResult.append(dist)
    fnum+=1


Non-Normalized Data ,Getting Distance arrays...
For fold 1 , prediction time = 127.39372944831848 
For fold 2 , prediction time = 127.56444835662842 
For fold 3 , prediction time = 127.68071365356445 
For fold 4 , prediction time = 131.48431587219238 
For fold 5 , prediction time = 129.12363290786743 


In [99]:
#class list for part 1
classes=df["Personality"].unique()


#Average of 5-fold Cross Validation Results
def get_Avrgs(arr,n):
    res=[]
    #calculating each folds average results
    for i in range(5):
        tmp=[0.0]*n
        for k in range(n):
            for j in range(5): 
                tmp[k]+=arr[j][i][k]
        res.append(tmp)
        
    for i in range(5):
        for j in range(n):
            res[i][j]=round(res[i][j]/5,3)
    rN=[]
    rW=[]
    
    #getting values in table format for jupyter markdown
    for i in range(5):
        rN.append("Accuracy : %"+str(res[i][0])+"<br> Precision : %"+str(res[i][1])+
                      "<br> Recall : %"+str(res[i][2]))
        rW.append("Accuracy : %"+str(res[i][3])+"<br> Precision : %"+str(res[i][4])+
                      "<br> Recall : %"+str(res[i][5]))
    #return normal averages and weighted averages of each folds
    return rN,rW


#processing KNN for each k value and each fold 
#this function simply returns pandas Dataframe of average,precision and recall metrics table
#for both normal and weighted KNN
def get_KNN_Table_pt1(fold_result,folds):
    kArray=[1,3,5,7,9]
    fnum=1
    avrgNonNormalizd=[]
    resaN=[]
    resaW=[]
    for dists in fold_result:
        tmp=[]
        aN=[]
        aW=[]
        for k in kArray:
            predsNormal=kNN(dists,k)
            predsWeighted=kNN(dists,k,kNNType="Weighted")
            accuracyN, precisionN, recallN=get_statics(folds[fnum-1][3],predsNormal,classes)
            accuracyW, precisionW, recallW=get_statics(folds[fnum-1][3],predsWeighted,classes)
            aN.append("Accuracy : %"+str(round(accuracyN,3))+"<br> Precision : %"+str(round(precisionN,3))+
                      "<br> Recall : %"+str(round(recallN,3)))
            aW.append("Accuracy : %"+str(round(accuracyW,3))+"<br> Precision : %"+str(round(precisionW,3))+
                      "<br> Recall : %"+str(round(recallW,3)))
            tmp.append([accuracyN,precisionN,recallN,accuracyW,precisionW,recallW])
        avrgNonNormalizd.append(tmp)
        fnum+=1
        resaN.append(aN)
        resaW.append(aW)
        
    avrN,avrW=get_Avrgs(avrgNonNormalizd.copy(),6)
    resaN.append(avrN)
    resaW.append(avrW)
    
    #normal KNN Results as Dataframe
    df1=pd.DataFrame(resaN,index=["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average Of The Folds"],
                       columns=["k=1","k=3","k=5","k=7","k=9"])
    
    #Weighted KNN Results as Dataframe
    df2=pd.DataFrame(resaW,index=["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average Of The Folds"],columns=["k=1","k=3","k=5","k=7","k=9"])
    return df1,df2
                     
#getting results of KNN for Non-Normalized Data
df_normal,df_weighted=get_KNN_Table_pt1(foldsResult,folds)

In [17]:
#printing Non-Normalized and Non-Weighted KNN for report
#print(df_normal.to_markdown())

In [18]:
#printing Non-Normalized and Weighted KNN for report
#print(df_weighted.to_markdown())

In [85]:
normalized_x=normalization(x) #normalize the train set
foldsNormalized=kFold(normalized_x,y)
#First , getting all the distance arrays to process KNN
foldsResultNormalized=[]
fnum=1
print("Normalized Data ,Getting Distance arrays...")
for fold in foldsNormalized:
    st=time.time()
    dist=calculate_distances(fold[0],fold[2],fold[1])
    et=time.time()
    passed=et-st
    print("For fold {} , prediction time = {} ".format(fnum,passed))
    foldsResultNormalized.append(dist)
    fnum+=1


Normalized Data ,Getting Distance arrays...
For fold 1 , prediction time = 151.71541738510132 
For fold 2 , prediction time = 147.46635460853577 
For fold 3 , prediction time = 148.93905568122864 
For fold 4 , prediction time = 146.65138053894043 
For fold 5 , prediction time = 148.01044344902039 


In [102]:
#getting results of KNN for Normalized Data
normalized_df_normal,normalized_df_weighted=get_KNN_Table_pt1(foldsResultNormalized,foldsNormalized)

In [19]:
#printing Normalized and Non-Weighted KNN for report as markdown table
#print(normalized_df_normal.to_markdown())

In [20]:
#printing Normalized and Weighted KNN for report as markdown table
#print(normalized_df_weighted.to_markdown())

<h1 style="text-transform: uppercase;
           font-size:40px;
           color: #4d2c6d;
            ">PART 1: Personality Classification</h1>

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table For Non-normalized and Non-Weighted Data</h3>
           
|                      | k=1                                                            | k=3                                                            | k=5                                                            | k=7                                                            | k=9                                                            |
|:---------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|
| Fold 1               | Accuracy : %97.45<br> Precision : %97.486<br> Recall : %97.461 | Accuracy : %98.4<br> Precision : %98.403<br> Recall : %98.426  | Accuracy : %98.65<br> Precision : %98.642<br> Recall : %98.662 | Accuracy : %98.6<br> Precision : %98.596<br> Recall : %98.616  | Accuracy : %98.45<br> Precision : %98.444<br> Recall : %98.468 |
| Fold 2               | Accuracy : %97.35<br> Precision : %97.374<br> Recall : %97.367 | Accuracy : %98.15<br> Precision : %98.176<br> Recall : %98.139 | Accuracy : %98.6<br> Precision : %98.615<br> Recall : %98.622  | Accuracy : %98.6<br> Precision : %98.606<br> Recall : %98.62   | Accuracy : %98.55<br> Precision : %98.556<br> Recall : %98.574 |
| Fold 3               | Accuracy : %97.75<br> Precision : %97.765<br> Recall : %97.787 | Accuracy : %98.7<br> Precision : %98.699<br> Recall : %98.708  | Accuracy : %98.95<br> Precision : %98.939<br> Recall : %98.956 | Accuracy : %98.95<br> Precision : %98.941<br> Recall : %98.966 | Accuracy : %98.95<br> Precision : %98.941<br> Recall : %98.966 |
| Fold 4               | Accuracy : %97.4<br> Precision : %97.454<br> Recall : %97.391  | Accuracy : %98.55<br> Precision : %98.595<br> Recall : %98.536 | Accuracy : %98.65<br> Precision : %98.699<br> Recall : %98.633 | Accuracy : %98.8<br> Precision : %98.839<br> Recall : %98.783  | Accuracy : %98.9<br> Precision : %98.94<br> Recall : %98.882   |
| Fold 5               | Accuracy : %97.35<br> Precision : %97.38<br> Recall : %97.375  | Accuracy : %98.4<br> Precision : %98.376<br> Recall : %98.396  | Accuracy : %98.85<br> Precision : %98.821<br> Recall : %98.845 | Accuracy : %98.8<br> Precision : %98.773<br> Recall : %98.802  | Accuracy : %98.9<br> Precision : %98.888<br> Recall : %98.894  |
| Average Of The Folds | Accuracy : %97.46<br> Precision : %97.492<br> Recall : %97.476 | Accuracy : %98.44<br> Precision : %98.45<br> Recall : %98.441  | Accuracy : %98.74<br> Precision : %98.743<br> Recall : %98.744 | Accuracy : %98.75<br> Precision : %98.751<br> Recall : %98.758 | Accuracy : %98.75<br> Precision : %98.754<br> Recall : %98.757 |




<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table For Non-normalized and Weighted Data</h3>
           
|                      | k=1                                                            | k=3                                                            | k=5                                                            | k=7                                                            | k=9                                                            |
|:---------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|
| Fold 1               | Accuracy : %97.45<br> Precision : %97.486<br> Recall : %97.461 | Accuracy : %98.45<br> Precision : %98.451<br> Recall : %98.47  | Accuracy : %98.6<br> Precision : %98.591<br> Recall : %98.622  | Accuracy : %98.65<br> Precision : %98.646<br> Recall : %98.663 | Accuracy : %98.6<br> Precision : %98.593<br> Recall : %98.617  |
| Fold 2               | Accuracy : %97.35<br> Precision : %97.374<br> Recall : %97.367 | Accuracy : %98.25<br> Precision : %98.279<br> Recall : %98.262 | Accuracy : %98.6<br> Precision : %98.615<br> Recall : %98.625  | Accuracy : %98.6<br> Precision : %98.606<br> Recall : %98.627  | Accuracy : %98.6<br> Precision : %98.606<br> Recall : %98.624  |
| Fold 3               | Accuracy : %97.75<br> Precision : %97.765<br> Recall : %97.787 | Accuracy : %98.85<br> Precision : %98.838<br> Recall : %98.868 | Accuracy : %98.95<br> Precision : %98.939<br> Recall : %98.956 | Accuracy : %98.9<br> Precision : %98.882<br> Recall : %98.919  | Accuracy : %98.95<br> Precision : %98.941<br> Recall : %98.966 |
| Fold 4               | Accuracy : %97.4<br> Precision : %97.454<br> Recall : %97.391  | Accuracy : %98.75<br> Precision : %98.791<br> Recall : %98.734 | Accuracy : %98.8<br> Precision : %98.845<br> Recall : %98.782  | Accuracy : %98.85<br> Precision : %98.888<br> Recall : %98.831 | Accuracy : %98.9<br> Precision : %98.942<br> Recall : %98.882  |
| Fold 5               | Accuracy : %97.35<br> Precision : %97.38<br> Recall : %97.375  | Accuracy : %98.5<br> Precision : %98.487<br> Recall : %98.502  | Accuracy : %98.8<br> Precision : %98.779<br> Recall : %98.795  | Accuracy : %98.8<br> Precision : %98.773<br> Recall : %98.802  | Accuracy : %98.9<br> Precision : %98.888<br> Recall : %98.894  |
| Average Of The Folds | Accuracy : %97.46<br> Precision : %97.492<br> Recall : %97.476 | Accuracy : %98.56<br> Precision : %98.569<br> Recall : %98.567 | Accuracy : %98.75<br> Precision : %98.754<br> Recall : %98.756 | Accuracy : %98.76<br> Precision : %98.759<br> Recall : %98.768 | Accuracy : %98.79<br> Precision : %98.794<br> Recall : %98.796 |


<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table For Normalized and Non-Weighted Data</h3>


|                      | k=1                                                            | k=3                                                            | k=5                                                            | k=7                                                            | k=9                                                            |
|:---------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|
| Fold 1               | Accuracy : %96.0<br> Precision : %96.063<br> Recall : %96.01   | Accuracy : %97.6<br> Precision : %97.614<br> Recall : %97.58   | Accuracy : %98.15<br> Precision : %98.158<br> Recall : %98.133 | Accuracy : %98.4<br> Precision : %98.407<br> Recall : %98.428  | Accuracy : %98.2<br> Precision : %98.211<br> Recall : %98.228  |
| Fold 2               | Accuracy : %96.15<br> Precision : %96.139<br> Recall : %96.155 | Accuracy : %97.75<br> Precision : %97.783<br> Recall : %97.764 | Accuracy : %98.05<br> Precision : %98.056<br> Recall : %98.069 | Accuracy : %98.25<br> Precision : %98.251<br> Recall : %98.269 | Accuracy : %98.5<br> Precision : %98.499<br> Recall : %98.523  |
| Fold 3               | Accuracy : %96.45<br> Precision : %96.439<br> Recall : %96.401 | Accuracy : %97.95<br> Precision : %97.969<br> Recall : %97.948 | Accuracy : %98.6<br> Precision : %98.596<br> Recall : %98.625  | Accuracy : %98.8<br> Precision : %98.808<br> Recall : %98.811  | Accuracy : %98.6<br> Precision : %98.61<br> Recall : %98.616   |
| Fold 4               | Accuracy : %96.5<br> Precision : %96.596<br> Recall : %96.505  | Accuracy : %97.8<br> Precision : %97.871<br> Recall : %97.782  | Accuracy : %98.25<br> Precision : %98.311<br> Recall : %98.227 | Accuracy : %98.4<br> Precision : %98.468<br> Recall : %98.379  | Accuracy : %98.55<br> Precision : %98.601<br> Recall : %98.532 |
| Fold 5               | Accuracy : %95.75<br> Precision : %95.721<br> Recall : %95.822 | Accuracy : %97.75<br> Precision : %97.704<br> Recall : %97.765 | Accuracy : %98.3<br> Precision : %98.27<br> Recall : %98.295   | Accuracy : %98.4<br> Precision : %98.376<br> Recall : %98.391  | Accuracy : %98.35<br> Precision : %98.332<br> Recall : %98.36  |
| Average Of The Folds | Accuracy : %96.17<br> Precision : %96.192<br> Recall : %96.179 | Accuracy : %97.77<br> Precision : %97.788<br> Recall : %97.768 | Accuracy : %98.27<br> Precision : %98.278<br> Recall : %98.27  | Accuracy : %98.45<br> Precision : %98.462<br> Recall : %98.456 | Accuracy : %98.44<br> Precision : %98.451<br> Recall : %98.452 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table For Normalized and Weighted Data</h3>

|                      | k=1                                                            | k=3                                                            | k=5                                                            | k=7                                                            | k=9                                                            |
|:---------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|:---------------------------------------------------------------|
| Fold 1               | Accuracy : %96.0<br> Precision : %96.063<br> Recall : %96.01   | Accuracy : %97.75<br> Precision : %97.77<br> Recall : %97.721  | Accuracy : %98.2<br> Precision : %98.215<br> Recall : %98.197  | Accuracy : %98.45<br> Precision : %98.456<br> Recall : %98.465 | Accuracy : %98.15<br> Precision : %98.165<br> Recall : %98.166 |
| Fold 2               | Accuracy : %96.15<br> Precision : %96.139<br> Recall : %96.155 | Accuracy : %98.0<br> Precision : %98.016<br> Recall : %98.016  | Accuracy : %98.2<br> Precision : %98.215<br> Recall : %98.225  | Accuracy : %98.3<br> Precision : %98.301<br> Recall : %98.313  | Accuracy : %98.5<br> Precision : %98.499<br> Recall : %98.523  |
| Fold 3               | Accuracy : %96.45<br> Precision : %96.439<br> Recall : %96.401 | Accuracy : %98.1<br> Precision : %98.093<br> Recall : %98.085  | Accuracy : %98.55<br> Precision : %98.553<br> Recall : %98.578 | Accuracy : %98.8<br> Precision : %98.808<br> Recall : %98.818  | Accuracy : %98.75<br> Precision : %98.757<br> Recall : %98.769 |
| Fold 4               | Accuracy : %96.5<br> Precision : %96.596<br> Recall : %96.505  | Accuracy : %97.8<br> Precision : %97.862<br> Recall : %97.767  | Accuracy : %98.35<br> Precision : %98.405<br> Recall : %98.322 | Accuracy : %98.55<br> Precision : %98.609<br> Recall : %98.527 | Accuracy : %98.55<br> Precision : %98.61<br> Recall : %98.524  |
| Fold 5               | Accuracy : %95.75<br> Precision : %95.721<br> Recall : %95.822 | Accuracy : %98.05<br> Precision : %98.007<br> Recall : %98.054 | Accuracy : %98.45<br> Precision : %98.412<br> Recall : %98.442 | Accuracy : %98.55<br> Precision : %98.522<br> Recall : %98.554 | Accuracy : %98.55<br> Precision : %98.529<br> Recall : %98.536 |
| Average Of The Folds | Accuracy : %96.17<br> Precision : %96.192<br> Recall : %96.179 | Accuracy : %97.94<br> Precision : %97.95<br> Recall : %97.929  | Accuracy : %98.35<br> Precision : %98.36<br> Recall : %98.353  | Accuracy : %98.53<br> Precision : %98.539<br> Recall : %98.535 | Accuracy : %98.5<br> Precision : %98.512<br> Recall : %98.503  |

<h1 style="color: #4d2c6d;">Error Analysis for Classification</h1>
<ul>
  <li><p> When we run the KNN algorithm, we notice that there are some samples which are misclassified. For instance, labels of some samples predicted as "ESTJ"(The Supervisor) but their actual labels are "ESFJ"(The Provider). One of the reasons of this misclassification can be that attributes of these classes are very close to each other. In dataset, there are some samples which are labeled as "ESTJ" or "ESFJ" and their attribute values are quite similar. This causes misclassification. Another reason for misclassification is that there are some samples whose labels are same but their attributes are very different. These differences make it difficult to find the correct label. Also, there are notable number of misclassified samples for "INTP"(The Thinker) and "INFP"(The Idealist). Actually, most of the misclassified samples' labels are relatively close to each other like "ESTJ"(The Supervisor) and "ESFJ"(The Provider).</p>
</li>
  <li><p> The accuracy of the model decreases, if we use fewer training samples. Because, when the number of training samples increase, the model is well-trained. So, number of training samples are important for training. The accuracy of the model increases as the number of training samples are increases if training samples are randomly distributed. There is another factor that affects accuracy of the model which is k-value of KNN algorithm. When we examine the results of the KNN algorithms from the tables, we can see that since the accuracy, precision and recall metrics are slightly increasing with k-value, the performance of the model increases proportionally with k-value. However, there are some exceptions at some k-values. Accuracy of the model is higher when we use weighted KNN algorithm rather than normal KNN algorithm. Also, normalization is another parameter that affects accuracy. In this problem, normalization is inversely proportional with accuracy of the model. </p>
</li>

</ul>

In [62]:
#data exploring 
df2=pd.read_csv("energy_efficiency_data.csv")
df2.head()

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution,Heating_Load,Cooling_Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [63]:
arr_2 = np.array(df2).astype(dtype="float64")

arr_2=shuffle(arr_2) #shuffling the array
x_values = arr_2[:, :-2]

#normalized feature values
normalized_x_values=normalization(x_values)

y_heating_load = arr_2[:, -2:-1]
y_cooling_load = arr_2[:, -1:]
y_heating_load = y_heating_load.reshape(len(y_heating_load),)
y_cooling_load = y_cooling_load.reshape(len(y_cooling_load),)

In [64]:
#5-fold cross validation 
folds_heating=kFold(x_values,y_heating_load)
folds_cooling=kFold(x_values,y_cooling_load)
folds_heating_normalized=kFold(normalized_x_values,y_heating_load)
folds_cooling_normalized=kFold(normalized_x_values,y_cooling_load)

In [65]:
#KNN regression algorithm for part 2
def kNN_for_Regression(distances,k,kNNType="Normal"):
    predictions=[]
    #weighted KNN
    if kNNType=="Weighted":
        
        for d in distances:
            #get first k elements of the distance array
            results = [row[1] for row in d[:k]] 
            dists=[row[0] for row in d[:k]] 
            neigs={}
            
            pred=0.0 
            weight_sum=0.0 #sum of k weightes
            for i in range(len(results)):
                #for each distance value, get its weight as 1/d
                weight=1.0/dists[i] 
                weight_sum+=weight
                #for each closest sample, sum up pred value with weight * samples label value
                pred+=weight*results[i] 
            
            #our prediction
            pred=pred/weight_sum 
            predictions.append(pred)
           
    #normal KNN
    else:
        for d in distances:
            #get first k elements of the distance array
            results = [row[1] for row in d[:k]]
            #getting average of the closest sample labels 
            pred=float(sum(results)/k)
            predictions.append(pred)

    return predictions



In [66]:

#calculating distance arrays for Heating Load
folds_heating_result=[]
fnum=1
print("Non-normalized Data ,Getting Distance Arrays......")
for fold in folds_heating:
    st=time.time()
    dist=calculate_distances(fold[0],fold[2],fold[1])
    et=time.time()
    passed=et-st
    print("For fold {} , prediction time = {} ".format(fnum,passed))
    folds_heating_result.append(dist)
    fnum+=1

folds_heating_result_normalized=[]
fnum=1
print("Normalized Data ,Getting Distance Arrays...")
for fold in folds_heating_normalized:
    st=time.time()
    dist=calculate_distances(fold[0],fold[2],fold[1])
    et=time.time()
    passed=et-st
    print("For fold {} , prediction time = {} ".format(fnum,passed))
    folds_heating_result_normalized.append(dist)
    fnum+=1

Non-normalized Data ,Getting Distance Arrays......
For fold 1 , prediction time = 0.7201764583587646 
For fold 2 , prediction time = 0.7227578163146973 
For fold 3 , prediction time = 0.7140519618988037 
For fold 4 , prediction time = 0.7194526195526123 
For fold 5 , prediction time = 0.7174234390258789 
Normalized Data ,Getting Distance Arrays...
For fold 1 , prediction time = 0.7194654941558838 
For fold 2 , prediction time = 0.7171614170074463 
For fold 3 , prediction time = 0.7224252223968506 
For fold 4 , prediction time = 0.7191641330718994 
For fold 5 , prediction time = 0.7173745632171631 


In [67]:
#calculating distance arrays for Cooling Load
folds_cooling_result=[]
fnum=1
print("Non-normalized Data Prediction...")
for fold in folds_cooling:
    st=time.time()
    dist=calculate_distances(fold[0],fold[2],fold[1])
    et=time.time()
    passed=et-st
    print("For fold {} , prediction time = {} ".format(fnum,passed))
    folds_cooling_result.append(dist)
    fnum+=1

folds_cooling_result_normalized=[]
fnum=1
print("Non-normalized Data Prediction...")
for fold in folds_cooling_normalized:
    st=time.time()
    dist=calculate_distances(fold[0],fold[2],fold[1])
    et=time.time()
    passed=et-st
    print("For fold {} , prediction time = {} ".format(fnum,passed))
    folds_cooling_result_normalized.append(dist)
    fnum+=1

Non-normalized Data Prediction...
For fold 1 , prediction time = 0.724172830581665 
For fold 2 , prediction time = 0.719172477722168 
For fold 3 , prediction time = 0.7211592197418213 
For fold 4 , prediction time = 0.7241582870483398 
For fold 5 , prediction time = 0.7183687686920166 
Non-normalized Data Prediction...
For fold 1 , prediction time = 0.7201566696166992 
For fold 2 , prediction time = 0.7176048755645752 
For fold 3 , prediction time = 0.7188091278076172 
For fold 4 , prediction time = 0.7184183597564697 
For fold 5 , prediction time = 0.7180113792419434 


In [68]:
# a function that returns average of MAE values for each k value
def get_Avrgs_For_Mae(arr,n):
    res=[]
    for i in range(5):
        tmp=[0.0]*n
        for k in range(n):
            for j in range(5): 
                tmp[k]+=arr[j][i][k]
        res.append(tmp)
        
    for i in range(5):
        for j in range(n):
            res[i][j]="MAE :%"+str(round(res[i][j]/5,3))
       
    return [i[0] for i in res], [i[1] for i in res]

#processing KNN for each k value and each fold 
#this function simply returns pandas Dataframe of MAE metrics table
#for both normal and weighted KNN
def get_KNN_Table_pt2(fold_result,folds):
    kArray=[1,3,5,7,9]
    fnum=1
    avrgArray=[]
    resNormal=[]
    resWeighted=[]
    for dists in fold_result:
        tmp=[]
        tmpN=[]
        tmpW=[]
        for k in kArray:
            predsNormal=kNN_for_Regression(dists,k)
            predsWeighted=kNN_for_Regression(dists,k,kNNType="Weighted")
          
            maeNormal=mae(predsNormal,folds[fnum-1][3])
            maeWeighted=mae(predsWeighted,folds[fnum-1][3])
            tmp.append([maeNormal,maeWeighted])
            tmpN.append("MAE: %"+str(round(maeNormal,3)))
            tmpW.append("MAE: %"+str(round(maeWeighted,3)))
        
        avrgArray.append(tmp)
        fnum+=1
        resNormal.append(tmpN)
        resWeighted.append(tmpW)
    avrN,avrW=get_Avrgs_For_Mae(avrgArray,2)
    resNormal.append(avrN)
    resWeighted.append(avrW)
    
    #normal KNN Results as Dataframe
    df1=pd.DataFrame(resNormal,index=["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average Of The Folds"],
                       columns=["k=1","k=3","k=5","k=7","k=9"])
    
    #weighted KNN Results as Dataframe
    df2=pd.DataFrame(resWeighted,index=["Fold 1","Fold 2","Fold 3","Fold 4","Fold 5","Average Of The Folds"],
                       columns=["k=1","k=3","k=5","k=7","k=9"])
    return df1,df2


In [69]:
#KNN processing for each combinations
dt_heating_normal,dt_heating_weighted=get_KNN_Table_pt2(folds_heating_result,folds_heating)

normalized_heating_normal, normalized_heating_weighted = get_KNN_Table_pt2(folds_heating_result_normalized,folds_heating_normalized)

dt_cooling_normal,dt_cooling_weighted=get_KNN_Table_pt2(folds_cooling_result,folds_cooling)

normalized_cooling_normal, normalized_cooling_weighted = get_KNN_Table_pt2(folds_cooling_result_normalized,folds_cooling_normalized)

In [ ]:

#printing Heating Load KNN results for report as markdown table
"""
print("Table of Heating Load For Non-normalized and Non-Weighted Data\n")
print(dt_heating_normal.to_markdown()+"\n")

print("Table of Heating Load For Non-normalized and Weighted Data\n")
print(dt_heating_weighted.to_markdown()+"\n")

print("Table of Heating Load For Normalized and Non-Weighted Data\n")
print(normalized_heating_normal.to_markdown()+"\n")

print("Table of Heating Load For Normalized and Weighted Data\n")
print(normalized_heating_weighted.to_markdown()+"\n")
"""


In [ ]:
#printing Cooling Load KNN results for report as markdown table

"""
print("Table of Cooling Load For Non-normalized and Non-Weighted Data\n")
print(dt_cooling_normal.to_markdown()+"\n")

print("Table of Cooling Load For Non-normalized and Weighted Data\n")
print(dt_cooling_weighted.to_markdown()+"\n")

print("Table of Cooling Load For Normalized and Non-Weighted Data\n")
print(normalized_cooling_normal.to_markdown()+"\n")

print("Table of Cooling Load For Normalized and Weighted Data\n")
print(normalized_cooling_weighted.to_markdown()+"\n")
"""


<h1 style="text-transform: uppercase;
           font-size:40px;
           color: #4d2c6d;
            ">PART 2: Energy Efficiency Estimation from Data</h1>

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;"> Table of Heating Load For Non-normalized and Non-Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %2.667 | MAE: %1.832 | MAE: %1.7   | MAE: %1.905 | MAE: %2.099 |
| Fold 2               | MAE: %2.506 | MAE: %1.757 | MAE: %1.768 | MAE: %2.034 | MAE: %2.198 |
| Fold 3               | MAE: %2.588 | MAE: %1.732 | MAE: %1.566 | MAE: %1.927 | MAE: %2.079 |
| Fold 4               | MAE: %2.588 | MAE: %1.64  | MAE: %1.381 | MAE: %1.659 | MAE: %1.888 |
| Fold 5               | MAE: %2.636 | MAE: %1.678 | MAE: %1.488 | MAE: %1.742 | MAE: %2.011 |
| Average Of The Folds | MAE :%2.597 | MAE :%1.728 | MAE :%1.581 | MAE :%1.853 | MAE :%2.055 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table of Heating Load For Non-normalized and Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %2.667 | MAE: %2.412 | MAE: %2.292 | MAE: %2.326 | MAE: %2.376 |
| Fold 2               | MAE: %2.506 | MAE: %2.286 | MAE: %2.291 | MAE: %2.393 | MAE: %2.445 |
| Fold 3               | MAE: %2.588 | MAE: %2.302 | MAE: %2.182 | MAE: %2.303 | MAE: %2.357 |
| Fold 4               | MAE: %2.588 | MAE: %2.252 | MAE: %2.055 | MAE: %2.116 | MAE: %2.203 |
| Fold 5               | MAE: %2.636 | MAE: %2.269 | MAE: %2.12  | MAE: %2.162 | MAE: %2.277 |
| Average Of The Folds | MAE :%2.597 | MAE :%2.304 | MAE :%2.188 | MAE :%2.26  | MAE :%2.332 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table of Heating Load For Normalized and Non-Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %3.571 | MAE: %2.186 | MAE: %2.131 | MAE: %2.014 | MAE: %1.97  |
| Fold 2               | MAE: %3.334 | MAE: %2.039 | MAE: %2.115 | MAE: %2.037 | MAE: %2.033 |
| Fold 3               | MAE: %2.741 | MAE: %1.686 | MAE: %1.665 | MAE: %1.604 | MAE: %1.605 |
| Fold 4               | MAE: %2.569 | MAE: %1.813 | MAE: %1.595 | MAE: %1.56  | MAE: %1.585 |
| Fold 5               | MAE: %3.324 | MAE: %2.07  | MAE: %2.006 | MAE: %1.925 | MAE: %1.966 |
| Average Of The Folds | MAE :%3.108 | MAE :%1.959 | MAE :%1.902 | MAE :%1.828 | MAE :%1.832 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table of Heating Load For Normalized and Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %3.571 | MAE: %2.151 | MAE: %2.11  | MAE: %2.02  | MAE: %1.972 |
| Fold 2               | MAE: %3.334 | MAE: %1.993 | MAE: %2.066 | MAE: %2.003 | MAE: %2.002 |
| Fold 3               | MAE: %2.741 | MAE: %1.675 | MAE: %1.638 | MAE: %1.595 | MAE: %1.581 |
| Fold 4               | MAE: %2.569 | MAE: %1.748 | MAE: %1.579 | MAE: %1.54  | MAE: %1.539 |
| Fold 5               | MAE: %3.324 | MAE: %2.021 | MAE: %1.969 | MAE: %1.919 | MAE: %1.931 |
| Average Of The Folds | MAE :%3.108 | MAE :%1.917 | MAE :%1.873 | MAE :%1.815 | MAE :%1.805 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table of Cooling Load For Non-normalized and Non-Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %2.201 | MAE: %1.323 | MAE: %1.418 | MAE: %1.588 | MAE: %1.714 |
| Fold 2               | MAE: %2.16  | MAE: %1.487 | MAE: %1.496 | MAE: %1.707 | MAE: %1.834 |
| Fold 3               | MAE: %2.257 | MAE: %1.522 | MAE: %1.4   | MAE: %1.658 | MAE: %1.829 |
| Fold 4               | MAE: %2.071 | MAE: %1.571 | MAE: %1.443 | MAE: %1.751 | MAE: %1.953 |
| Fold 5               | MAE: %2.125 | MAE: %1.533 | MAE: %1.329 | MAE: %1.557 | MAE: %1.766 |
| Average Of The Folds | MAE :%2.163 | MAE :%1.488 | MAE :%1.417 | MAE :%1.652 | MAE :%1.819 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table of Cooling Load For Non-normalized and Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %2.201 | MAE: %1.842 | MAE: %1.738 | MAE: %1.741 | MAE: %1.785 |
| Fold 2               | MAE: %2.16  | MAE: %1.779 | MAE: %1.76  | MAE: %1.799 | MAE: %1.859 |
| Fold 3               | MAE: %2.257 | MAE: %1.939 | MAE: %1.803 | MAE: %1.871 | MAE: %1.931 |
| Fold 4               | MAE: %2.071 | MAE: %1.864 | MAE: %1.771 | MAE: %1.878 | MAE: %1.98  |
| Fold 5               | MAE: %2.125 | MAE: %1.816 | MAE: %1.678 | MAE: %1.737 | MAE: %1.812 |
| Average Of The Folds | MAE :%2.163 | MAE :%1.848 | MAE :%1.75  | MAE :%1.805 | MAE :%1.873 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table of Cooling Load For Normalized and Non-Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %3.811 | MAE: %2.355 | MAE: %2.334 | MAE: %2.208 | MAE: %2.191 |
| Fold 2               | MAE: %3.77  | MAE: %2.444 | MAE: %2.497 | MAE: %2.334 | MAE: %2.279 |
| Fold 3               | MAE: %2.911 | MAE: %1.933 | MAE: %1.911 | MAE: %1.837 | MAE: %1.786 |
| Fold 4               | MAE: %2.834 | MAE: %1.918 | MAE: %1.874 | MAE: %1.783 | MAE: %1.816 |
| Fold 5               | MAE: %3.613 | MAE: %2.223 | MAE: %2.158 | MAE: %1.994 | MAE: %1.99  |
| Average Of The Folds | MAE :%3.388 | MAE :%2.175 | MAE :%2.155 | MAE :%2.031 | MAE :%2.013 |

<h3 style="text-align:center ;text-transform: uppercase;
           color: #4d2c6d;">Table of Cooling Load For Normalized and Weighted Data</h3>

|                      | k=1         | k=3         | k=5         | k=7         | k=9         |
|:---------------------|:------------|:------------|:------------|:------------|:------------|
| Fold 1               | MAE: %3.811 | MAE: %2.354 | MAE: %2.337 | MAE: %2.237 | MAE: %2.215 |
| Fold 2               | MAE: %3.77  | MAE: %2.427 | MAE: %2.474 | MAE: %2.35  | MAE: %2.307 |
| Fold 3               | MAE: %2.911 | MAE: %1.937 | MAE: %1.903 | MAE: %1.852 | MAE: %1.802 |
| Fold 4               | MAE: %2.834 | MAE: %1.881 | MAE: %1.853 | MAE: %1.785 | MAE: %1.793 |
| Fold 5               | MAE: %3.613 | MAE: %2.204 | MAE: %2.151 | MAE: %2.02  | MAE: %2.016 |
| Average Of The Folds | MAE :%3.388 | MAE :%2.161 | MAE :%2.144 | MAE :%2.049 | MAE :%2.026 |

 
<h1 style="color: #4d2c6d;">Error Analysis for Regression </h1>
<ul>
  <li><p> There are two distinct ground-truth values which named "Heating Load" and "Cooling Load" in this regression problem. Number of training samples are significant factor for the accuracy of the model. Using fewer number of training samples affects accuracy of the model negatively. For both ground-truth values until a certain k-value mean absolute error decreases. However, if k-value keeps increasing mean absolute error increases. So, accuracy of the model and k-value are not proportional. Increasing the k-value may not be a solution all the time for better accuracy of the model. When we investigate the average folds results from the tables, we realize that normalization affects ground-truth values differently. For "Heating Load" ground-truth value, mean absolute error decreases when we use normalization but for "Cooling Load" ground-truth value, mean absolute error increases that means accuracy of the model decreases when we use normalization. However, if we use weighted KNN algorithm instead of normal KNN algorithm, accuracy of the model increases for both ground-truth values. </p>

</ul>